### 打开关闭窗口

In [2]:
import cv2
#创建窗口
cv2.namedWindow('windows',cv2.WINDOW_NORMAL)
#修改窗口大小
cv2.resizeWindow('windows',800,600)
#展示名字为window的窗口
cv2.imshow('windows',0)
#等待按键输入，最多delay 5000ms
key = cv2.waitKey(0)
#key 是int型，最少是16位，但是ascii码是8位
if key & 0xFF == ord('q'):
    print("exit")
    cv2.destroyAllWindows()

exit


### 读取图片并打开

In [18]:
def cv_show(name,img):
    cv2.imshow(name,img)
    key = cv2.waitKey(0)
    if key == ord('q'):
        cv2.destroyAllWindows()

#读相对目录下的某个照片
cat = cv2.imread('./picture/cat.jpeg')

#窗口打开图片
cv_show('cat',cat)

### 保存图片

In [30]:
def cv_catch(num,name):
    str_num = str(num)
    catalog = f"""./picture/{str_num}.png"""
    #print(catalog)
    cv2.imwrite(catalog,name)

In [ ]:
cv2.imshow('cat',cat)
num = 0
while True:
    key = cv2.waitKey(0)
    #每次按下's'键后，保存图片并num+1
    if key == ord('s'):       
        cv_catch(num,cat)
        num += 1 
    #每次按下'q'键后，退出所有窗口
    elif key == ord('q'):
        num = 0
        cv2.destroyAllWindows()
        break
    

    

### 视频采集

In [4]:
#打开目录的指定视频

cv2.namedWindow('video',cv2.WINDOW_NORMAL)


#打开摄像头
#cap = cv2.VideoCapture(0)
#cv2.resizeWindow('video',640,480)

#打开视频
cap = cv2.VideoCapture('./video/koen.mp4')
cv2.resizeWindow('video',720,1280)

#循环读每一帧
while True:
    #读一帧，返回标记
    ret, frame = cap.read()
    #根据ret判断
    if not ret:
        break
    #显示数据
    cv2.imshow('video',frame)
    
    #隔100ms检测一下
    key = cv2.waitKey(16)
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    cap.release
    


### 识别视频中的人脸

In [10]:
cap = cv2.VideoCapture(0)
cv2.namedWindow('video',cv2.WINDOW_NORMAL)
cv2.resizeWindow('video',640,480)
#导入特征文件
face_detector = cv2.CascadeClassifier('./classfile/haarcascade_frontalface_alt.xml')

while True:
    #ret为是否读取标志位
    ret,frame = cap.read() 
     #r如果读不到，则退出和释放
    if not ret:
        break
    #得到灰阶图
    gray = cv2.cvtColor(frame, code=cv2.COLOR_BGR2GRAY)
    #检测人脸
    faces = face_detector.detectMultiScale(gray)
    for x,y,w,h in faces:
        cv2.rectangle(frame,
                      pt1 = (x,y),
                      pt2=(x+w,y+h),
                      color=[0,0,255],thickness=2
                      )
    cv2.imshow('video',frame)
    key = cv2.waitKey(1000//60)
    #键入Q则关闭窗口和释放
    if key == ord('q'): 
        break
    
cv2.destroyAllWindows()
cap.release()
    
